In [ ]:
from ngsolve import *
import time

from ngsolve.meshes import *
#from ngsolve.solvers import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square

import numpy as np
#import matplotlib.pyplot as plt

#from su2_yangmills import *
from lgt import *



In [ ]:
from pathlib import Path
from ngsolve.fem import CompilePythonModule

lgtcpp_path = "../../src/gaugelinkfe/lgt.cpp" 

if True:
    print("compile it")
    m = CompilePythonModule(Path(lgtcpp_path), init_function_name="lgt_module")
else:
    print("load it")
    #import lgt_module as m

In [ ]:
ne=100
#mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=True, periodic_y=True)
mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#mesh = MakeStructured2DMesh(quads=False, nx=ne, ny=ne, periodic_x=False, periodic_y=False)

#mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

#mesh = unit_square.GenerateMesh(maxh=0.1)
Draw(mesh)
dt = 0.005
meshvars = m.MeshVars(mesh, dt)

In [ ]:
#draw_tbone_weights(meshvars)
#draw_bone_weights(meshvars)

In [ ]:
order=0
#fesC = L2(mesh, order=order)**3 # for su2 charge
#fesJ = HDiv(mesh, order=order, discontinuous=True)**3 # for su2 charge current

# discrete connection one form sitting on borders
#fesA = FacetFESpace(mesh, order=0)**3

# parallel transport maps between neighbouring volumes, belonging to an oriented interface
#TODO: orientation?
fesU = FacetFESpace(mesh, order=0)**4


n=1
fesA = HCurl(mesh, order=3)**3

# space of wilson loops centered around bones (= corners (2D) or edges (3D))
# for visualization only!
fesW = H1(mesh, order=order)**4


# space of magnetic energy
fesHB = H1(mesh, order=order)

# space of electric energy
fesHE = HCurl(mesh, order=order)

# space of current flux
fesjflux = FacetFESpace(mesh, order=0)**3

In [ ]:
gfU = GridFunction(fesU)
random_links(gfU, qthetamax=0.01*pi, thetamax=0.1*pi, phimax=0.1*pi)

gfHB = GridFunction(fesHB)
gfHB.vec.data[:] = 0.
m.CalcgfHB(meshvars,gfU,gfHB)
#Draw(gfHB, mesh)


gfU0 = GridFunction(fesU)
q = np.array([1.,0. ,0. ,0.])
set_gfU_qconst(gfU0, q)


gfHE = GridFunction(fesHE)
gfHE.vec.data[:] = 0.
m.CalcgfHE(meshvars,gfU0,gfU,gfHE)
#Draw(gfHE, mesh)

In [ ]:
mu = [0.5, 0.5]
sigma2 = 0.1
r_u =  [x - mu[0], y - mu[1]]
rot_u =  [y - mu[1],x - mu[0]]

def gaussxy(mu, sigma2):
    return exp(-0.5*((x-mu[0])*(x-mu[0]) + (y-mu[1])*(y-mu[1]))/sigma2)


signx = IfPos(x - mu[0], 1, -1)
signy = IfPos(y - mu[1], 1, -1)

posy = IfPos(y - mu[1], 1, 0)
posx = IfPos(x - mu[0], 1, 0)

A0 = 10

#CFA = CF( (A0*posy*posx,0,0,0 ,0,0) )
#CFA = CF( (0,A0*posy*posx,0,0 ,0,0) )

CFA = CF( (A0*signy,0, 0,0 ,0,0) )


gfA = GridFunction(fesA)

gfA.Set(CFA)

#Draw(CFA, mesh)
Draw(Norm(gfA), mesh)

gfU = GridFunction(fesU)
m.InterpolateAtoU(meshvars, gfA, gfU)

Enum = 5
U = m.GetLink(gfU, Enum, True)
print(f"U[{Enum}]")
print(U)

gfHB = GridFunction(fesHB)
m.CalcgfHB(meshvars,gfU, gfHB)
Draw(gfHB, mesh)

In [ ]:
gfU0 = GridFunction(fesU)
q = np.array([1.,0. ,0. ,0.])
set_gfU_qconst(gfU0, q)

gfU = GridFunction(fesU)

# TODO: set meaningful initial conditions 
eps = 0.3
q = np.array([1.,0. ,0. ,0.])
set_gfU_qconst(gfU, q)

# ne=10
# Enum = 95
# set_gfU_link(gfU, Enum, qtheta=eps*pi, theta=0., phi=0., glob_or=True)
# set_gfU_link(gfU, Enum+20, qtheta=-eps*pi, theta=0., phi=0., glob_or=True)
# set_gfU_link(gfU, Enum+21, qtheta=-eps*pi, theta=0., phi=0., glob_or=True)
# set_gfU_link(gfU, Enum+18, qtheta=eps*pi, theta=0., phi=0., glob_or=True)

# ne=100
#Enum = 2480
#Enum = 2472
#Enum = 2473
Enum = 2473

Vnum = 0
Enums = []

for v in mesh.vertices:
    if( np.linalg.norm( np.array(v.point) - np.array([0.5, 0.5]) ) < 0.00001 ):
        #print(v.nr)
        #print(v.point)
        for e in v.edges:
            Enums.append(e.nr)

#print(Enums)

# IC FOR SUQARE MESH
#set_gfU_link(gfU, Enums[0], qtheta=eps*pi, theta=0., phi=0., glob_or=True)
#set_gfU_link(gfU, Enums[1], qtheta=eps*pi, theta=0., phi=0., glob_or=True)
#set_gfU_link(gfU, Enums[2], qtheta=-eps*pi, theta=0., phi=0., glob_or=True)
#set_gfU_link(gfU, Enums[3], qtheta=-eps*pi, theta=0., phi=0., glob_or=True)


gfHB = GridFunction(fesHB)
gfHB.vec.data[:] = 0.
m.CalcgfHB(meshvars,gfU,gfHB)
Draw(gfHB, mesh)


gfHE = GridFunction(fesHE)
#gfHE.vec.data[:] = 0.
m.CalcgfHE(meshvars,gfU0,gfU,gfHE)
Draw(gfHE, mesh)

In [ ]:
feshc = HCurl(mesh, order=0)
mhc = BilinearForm(feshc)
uhc,vhc = feshc.TnT()
mhc += InnerProduct(uhc,vhc)*dx
mhc.Assemble()
mhcmat = mhc.mat


fesh1 = H1(mesh, order=1)
mh1 = BilinearForm(fesh1)
uh1,vh1 = fesh1.TnT()
mh1 += InnerProduct(uh1,vh1)*dx
mh1.Assemble()
mh1mat = mh1.mat


#a0 = dt
#ai = 1./ne
#a0i = a0*ai
#aij = ai*ai
#print(ai)

M = mhcmat
Mt = mh1mat

Mee = M[0,0]
Metet = Mt[0,0]

print(Mee)
print(Metet)
print(1./Metet)

In [ ]:
#gfU = GridFunction(fesU)
gfUnew = GridFunction(fesU)
gfUold = GridFunction(fesU)

gfUold.vec.data[:] = gfU.vec.data[:]
gfUnew.vec.data[:] = gfU.vec.data[:]



gfHB = GridFunction(fesHB)
gfHE = GridFunction(fesHE)
m.CalcgfHB(meshvars,gfU,gfHB)
m.CalcgfHE(meshvars,gfUold,gfU,gfHE)
gfHBscene = Draw(gfHB, mesh)
gfHEscene = Draw(gfHE, mesh)


fesl2 = L2(mesh, order=0)
gfgausserr2 = GridFunction(fesl2)
#gfgausserr2.vec.data[:] = 0.
err2scene = Draw(gfgausserr2, mesh)


gfj_flux = GridFunction(fesjflux)
gfj_flux.vec.data[:] = 0.

# caclulate integrated charge per element
gfrhoint = GridFunction(L2(mesh, order=0)**3)
gfrhoint.vec.data[:] = 0.

scenes = [gfHBscene, gfHEscene, err2scene]

def gausserr2(ne, eps, gfgausserr2, scene):
    for elnum in range(ne*ne):
        #print ("\r", f"elnum={elnum}", end="")
        gfgausserr2.vec.data[elnum] = m.GaussElementCheck(meshvars, gfU, gfUnew, gfrhoint, elnum, eps)
        #time.sleep(1)
    scene.Redraw()

eps = 0.00000001
gausserr2(ne, eps, gfgausserr2, err2scene)

# required for python code
#mesh_oriented_vedges = []
#for v in mesh.vertices:
#    #print(v)
#    #print(type(v))
#    mesh_oriented_vedges.append(get_oriented_vedges_tan(v, mesh))
#v_weights = [Mt[_v_id, _v_id] for _v_id in range(mesh.nv)]
#e_weights = [M[_e_id, _e_id] for _e_id in range(mesh.nedge)]

#dt = 0.1
#dt = 0.1
#dt = 0.5
t = 15
#dt = 0.001
#dt = 0.0001

n_steps = int(t/dt)
    
    
meshvars.EvaluateFacetVorb()
# TODO: correct initial conditions

folder_name = "lgt_fem_eom"
parameters = {}
parameters['n'] = ne
parameters['order'] = 1
parameters['dt'] = dt
parfolder_name = f"{folder_name}/{parameters['n']}_order{parameters['order']}_dt{parameters['dt']}"  
if not os.path.exists(parfolder_name):
    os.makedirs(parfolder_name)

snap_names = ["H_{B}", "H_{E}", "GaussErr2"]

if not os.path.exists(f"{parfolder_name}/vtk"):
    os.makedirs(f"{parfolder_name}/vtk")
vtk = VTKOutput(mesh, coefs=[gfHB, gfHE, gfgausserr2], names=snap_names,filename=f"{parfolder_name}/vtk/vtk_snap",subdivision=2)
vtk.Do(time=0)
nt_snaps = list(range(0,n_steps,int(n_steps/15)))


# input()
for i in range(n_steps):
#for i in []:
    
    ### python function
    #py_timestep(gfU, gfUnew, gfUold, mesh)
    #py_timestep(gfU, gfUnew, gfUold, mesh, v_weights, e_weights, dt, vedges_list=mesh_oriented_vedges)
    
    ### cpp function
    m.EOMUpdate(meshvars, gfU, gfUold, gfUnew, gfj_flux, dt)
    
    gfUold.vec.data[:] = gfU.vec.data[:]
    gfU.vec.data[:] = gfUnew.vec[:]
    
    ### python function
    #calc_gfW(gfU, gfW, mesh)
    #calc_wilson_action(gfW,gfHB)
    
    ### cpp function
    m.CalcgfHB(meshvars,gfU,gfHB)
    m.CalcgfHE(meshvars,gfUold,gfU,gfHE)
    
    if i % 1 == 0:
        gausserr2(ne, eps, gfgausserr2, err2scene)
        gausserr2norm = Integrate(Norm(gfgausserr2), mesh)
        if(gausserr2norm > eps):
            raise ValueError(f"Gauß constraint is violated:{gausserr2norm}")
        for scene in scenes:
            scene.Redraw()

    if (i in nt_snaps):
        current_snaps = {}
        for snap_name in snap_names:
            pass
            # save vtk snapshots
            vtk.Do(time=i*dt)
            #vtk.Do(time=t)
    
    #input()
    #time.sleep(0.5)
    print ("\r", f"timestep:{i}", end="")
